In [14]:
!pip install pyomo
!apt-get install -y coinor-libhighs-dev
!apt-get install -y glpk-utils
!pip install amplpy pyomo -q
!python -m amplpy.modules install coin highs scip gcg -q

"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [15]:
import pandas as pd
import math
import requests
import networkx as nx
import matplotlib.pyplot as plt
from pyomo.environ import value, SolverFactory
import numpy as np

# Def clases

In [16]:
class Vehicle:
    def __init__(self, name, vehicle_type, freight_rate_km, time_rate_min, daily_maintenance, fuel_cost,
                 fuel_time, avg_speed_kmh, fuel_efficiency_km_gal, capacity,
                 electricity_efficiency_kwh_km=None, range_km=None):
        self.name = name
        self.vehicle_type = vehicle_type  # Agregar este atributo
        self.freight_rate_km = freight_rate_km
        self.time_rate_min = time_rate_min
        self.daily_maintenance = daily_maintenance
        self.fuel_cost = fuel_cost
        self.fuel_time = fuel_time
        self.avg_speed_kmh = avg_speed_kmh
        self.fuel_efficiency_km_gal = fuel_efficiency_km_gal
        self.capacity = capacity
        self.electricity_efficiency_kwh_km = electricity_efficiency_kwh_km
        self.range_km = range_km

    def calculate_cost(self, i, j, x, car_matrix_distance, car_matrix_time, drone_matrix_distance, num_depots):
        cost = 0
        if self.vehicle_type == "Gas Car":
            distance = car_matrix_distance[i-1][j-1]
            cost += distance * x * self.freight_rate_km
            cost += car_matrix_time[i-1][j-1] * x * self.time_rate_min
            cost += (distance / self.fuel_efficiency_km_gal) * self.fuel_cost * x
        elif self.vehicle_type == "Drone":
            distance = drone_matrix_distance[i-1][j-1]
            if self.avg_speed_kmh:
                time = distance / self.avg_speed_kmh * 60  # Convertir a minutos
            else:
                time = 0
            cost += distance * x * self.freight_rate_km
            cost += time * x * self.time_rate_min
            if self.electricity_efficiency_kwh_km:
                cost += distance * self.electricity_efficiency_kwh_km * self.fuel_cost * x
        else:
            print(f"Tipo de vehículo no reconocido: {self.vehicle_type}")
        return cost

    def calculate_distance(self, i, j, car_matrix_distance, drone_matrix_distance):
        if self.vehicle_type == "Gas Car":
            return car_matrix_distance[i-1][j-1]
        elif self.vehicle_type == "Drone":
            return drone_matrix_distance[i-1][j-1]
        else:
            return 0

class GasCar(Vehicle):
    def __init__(self, **kwargs):
        super().__init__(vehicle_type="Gas Car", **kwargs)

class Drone(Vehicle):
    def __init__(self, **kwargs):
        super().__init__(vehicle_type="Drone", **kwargs)
        
class Depot:
    def __init__(self, location_id, longitude, latitude,capacity):
        self.location_id = location_id
        self.longitude = longitude
        self.latitude = latitude
        self.capacity = capacity

class Client:
    def __init__(self, location_id, longitude, latitude, product):
        self.location_id = location_id
        self.longitude = longitude
        self.latitude = latitude
        self.product = product  # Demanda específica

    def __str__(self):
        return f"Client(location_id={self.location_id}, longitude={self.longitude}, latitude={self.latitude}, product={self.product})"

def load_depots_with_capacity(depots_csv_path, depot_capacities_csv_path):
    # Cargar datos de los depósitos y las capacidades
    depots_df = pd.read_csv(depots_csv_path)
    capacities_df = pd.read_csv(depot_capacities_csv_path)

    # Unir ambas tablas usando DepotID como clave
    merged_df = pd.merge(depots_df, capacities_df, on="DepotID", how="left")

    # Crear una lista de objetos Depot
    depots = []
    for _, row in merged_df.iterrows():
        depot = Depot(
            location_id=row['LocationID'],
            longitude=row['Longitude'],
            latitude=row['Latitude'],
            capacity=row['Product'] if not pd.isna(row['Product']) else 0  # Si falta capacidad, asumir 0
        )
        depots.append(depot)
    return depots

def load_vehicles(csv_path):
    vehicles_df = pd.read_csv(csv_path)
    vehicles = []
    for index, row in vehicles_df.iterrows():
        vehicle_type = row['VehicleType'].strip().lower()
        capacity = row['Capacity']
        range_km = row['Range']
        if vehicle_type == 'gas car':
            vehicle = GasCar(
                name=f"Gas Car V{index+1}",
                freight_rate_km=5000,
                time_rate_min=500,
                daily_maintenance=30000,
                fuel_cost=16000,
                fuel_time=0.1,
                avg_speed_kmh=None,
                fuel_efficiency_km_gal=10,
                capacity=capacity,
                range_km=range_km
            )
            vehicles.append(vehicle)
        elif vehicle_type == 'drone':
            vehicle = Drone(
                name=f"Drone V{index+1}",
                freight_rate_km=500,
                time_rate_min=500,
                daily_maintenance=3000,
                fuel_cost=220.73,
                fuel_time=2,
                avg_speed_kmh=40,
                fuel_efficiency_km_gal=None,
                capacity=capacity,
                electricity_efficiency_kwh_km=0.15,
                range_km=range_km
            )
            vehicles.append(vehicle)
        else:
            print(f"Tipo de vehículo desconocido: {row['VehicleType']}. Se omite.")
    return vehicles



def validate_data(depots_df, clients_df,capacities_df):
    required_depot_columns = ['LocationID', 'Longitude', 'Latitude']
    required_capacity_columns = ['DepotID', 'Product']
    required_client_columns = ['LocationID', 'Longitude', 'Latitude', 'Product']

    if not set(required_depot_columns).issubset(depots_df.columns):
        raise ValueError("El archivo DepotsMini.csv no contiene las columnas requeridas.")

    if not set(required_client_columns).issubset(clients_df.columns):
        raise ValueError("El archivo ClientsMini.csv no contiene las columnas requeridas.")

    if not set(required_capacity_columns).issubset(capacities_df.columns):
        raise ValueError("El archivo depotCapacities.csv no contiene las columnas requeridas.")

    # Verificar valores faltantes
    if depots_df[required_depot_columns].isnull().any().any():
        raise ValueError("El archivo DepotsMini.csv contiene valores faltantes.")

    if clients_df[required_client_columns].isnull().any().any():
        raise ValueError("El archivo ClientsMini.csv contiene valores faltantes.")

"""
depots_csv_path = '/content/Depots.csv'
depot_capacities_csv_path = '/content/DepotCapacities.csv'

depots_set = load_depots_with_capacity(depots_csv_path, depot_capacities_csv_path)

# Imprimir información básica de los depósitos cargados
for depot in depots_set:
    print(f"Depot ID: {depot.location_id}, Longitud: {depot.longitude}, Latitud: {depot.latitude}, Capacidad: {depot.capacity}")
"""



'\ndepots_csv_path = \'/content/Depots.csv\'\ndepot_capacities_csv_path = \'/content/DepotCapacities.csv\'\n\ndepots_set = load_depots_with_capacity(depots_csv_path, depot_capacities_csv_path)\n\n# Imprimir información básica de los depósitos cargados\nfor depot in depots_set:\n    print(f"Depot ID: {depot.location_id}, Longitud: {depot.longitude}, Latitud: {depot.latitude}, Capacidad: {depot.capacity}")\n'

## Modelo

In [ ]:
# Bloque 2: Definición del Modelo de Optimización

import numpy as np
import requests
from pyomo.environ import *
from pyomo.environ import SolverFactory
from pyomo.environ import value
from amplpy import modules

class TransportationModel:
    def __init__(self, vehicles, depots, clients):
        self.vehicles = vehicles
        self.depots = depots
        self.clients = clients

        # Imprimir el cliente 1 de manera legible
        print(f"Cliente 1: {self.clients[0]}")


        self.all_coords = [(depot.longitude, depot.latitude) for depot in depots] + \
                          [(client.longitude, client.latitude) for client in clients]

        self.index_to_location_id = {}
        self.location_id_to_index = {}
        idx = 1

        for depot in depots:
            self.index_to_location_id[idx] = depot.location_id
            self.location_id_to_index[depot.location_id] = idx
            idx += 1
        for client in clients:
            self.index_to_location_id[idx] = client.location_id
            self.location_id_to_index[client.location_id] = idx
            idx += 1

        self.car_matrix_distance = None
        self.car_matrix_time = None
        self.drone_matrix_distance = None
        self.drone_matrix_time = None
        self.model = ConcreteModel()

    def build_distance_matrices(self):
        # Construir cadenas de coordenadas para OSRM
        coords_str = ';'.join([f"{lon},{lat}" for lon, lat in self.all_coords])

        # URL de la API OSRM
        url = f"https://router.project-osrm.org/table/v1/driving/{coords_str}"

        # Parámetros de la solicitud
        params = {
            'sources': ';'.join(map(str, range(len(self.all_coords)))),
            'destinations': ';'.join(map(str, range(len(self.all_coords)))),
            'annotations': 'duration,distance'
        }

        # Enviar la solicitud
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error al conectar con la API OSRM: {e}")
            exit()

        data = response.json()

        # Extraer matrices de distancia y tiempo
        self.car_matrix_distance = np.array(data['distances']) / 1000  # Convertir a km
        self.car_matrix_time = np.array(data['durations']) / 60         # Convertir a minutos

        # Calcular matriz para drones usando fórmula de Haversine
        self.drone_matrix_distance = self.calculate_haversine_matrix()
        self.drone_matrix_time = self.drone_matrix_distance / 40 * 60  # Asumiendo velocidad de 40 km/h

        # Rellenar la diagonal con un valor alto para evitar rutas de un nodo a sí mismo
        np.fill_diagonal(self.car_matrix_distance, 9999999)
        np.fill_diagonal(self.car_matrix_time, 9999999)
        np.fill_diagonal(self.drone_matrix_distance, 9999999)
        np.fill_diagonal(self.drone_matrix_time, 9999999)

    def calculate_haversine_matrix(self):
        def haversine(lon1, lat1, lon2, lat2):
            lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
            c = 2 * math.asin(math.sqrt(a))
            r = 6371  # Radio de la Tierra en km
            return c * r

        num_nodes = len(self.all_coords)
        matrix = np.zeros((num_nodes, num_nodes))

        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j:
                    origin = self.all_coords[i]
                    destination = self.all_coords[j]
                    matrix[i][j] = haversine(origin[0], origin[1], destination[0], destination[1])

        return matrix

    def build_model(self):
        model = self.model
        num_depots = len(self.depots)
        num_clients = len(self.clients)
        num_nodes = len(self.all_coords)
        num_cars = len(self.vehicles)

        # Verificar que el número de vehículos coincide
        assert num_cars == len(self.vehicles), "El número de vehículos no coincide con la lista de vehículos."

        # Conjuntos
        model.depots_set = RangeSet(1, num_depots)
        print(f"Depots_set definido: {[i for i in model.depots_set]}")
        model.clients_set = RangeSet(num_depots + 1, num_nodes)
        print(model.clients_set)
        model.nodes_set = RangeSet(1, num_nodes)
        model.cars_set = RangeSet(1, num_cars)

        # Variables de decisión
        model.x = Var(model.nodes_set, model.nodes_set, model.cars_set, domain=Binary)
        model.u = Var(model.clients_set, model.cars_set, domain=NonNegativeReals, bounds=(0, num_clients))

            # Para llevar cuenta de las ofertas
        model.y = Var(model.depots_set, model.cars_set, within=NonNegativeReals)
            # Para llevar cuenta de las demandas
        model.z = Var(model.clients_set, model.cars_set, within=NonNegativeReals)
        
        # Función Objetivo: Minimizar costos totales
        def objective_rule(model):
            cost = 0
            for k in model.cars_set:
                vehicle = self.vehicles[k-1]
                for i in model.nodes_set:
                    for j in model.nodes_set:
                        cost += vehicle.calculate_cost(i, j, model.x[i, j, k],
                                                        self.car_matrix_distance,
                                                        self.car_matrix_time,
                                                        self.drone_matrix_distance,
                                                        num_depots)
            # Agregar costos de mantenimiento
            for k in model.cars_set:
                vehicle = self.vehicles[k-1]
                cost += vehicle.daily_maintenance
            return cost
        model.obj = Objective(rule=objective_rule, sense=minimize)
        
        # =============================
        # Restricciones
        # =============================
        
        # 1. Conservación de flujo para clientes
        def flow_conservation_rule(model, j, k):
            return sum(model.x[i, j, k] for i in model.nodes_set) - sum(model.x[j, i, k] for i in model.nodes_set) == 0
        model.flow_conservation = Constraint(model.clients_set, model.cars_set, rule=flow_conservation_rule)
        
        # 2. Cada cliente es visitado exactamente una vez
        def visit_once_rule(model, j):
            return sum(model.x[i, j, k] for i in model.nodes_set for k in model.cars_set) == 1
        model.visit_once = Constraint(model.clients_set, rule=visit_once_rule)

        # 3. Cada vehículo comienza y termina en un depósito
        def start_depot_rule(model, k):
            return sum(model.x[i, j, k] for i in model.depots_set for j in model.nodes_set) == 1
        model.start_depot = Constraint(model.cars_set, rule=start_depot_rule)

        def end_depot_rule(model, k):
            return sum(model.x[i, j, k] for j in model.nodes_set for i in model.depots_set) == 1
        model.end_depot = Constraint(model.cars_set, rule=end_depot_rule)

        # 4. Restricción de capacidad de los vehículos
        def capacity_rule(model, k):
            vehicle = self.vehicles[k-1]
            return sum(self.clients[j - num_depots - 1].product * model.x[i, j, k]
                       for i in model.nodes_set
                       for j in model.clients_set) <= vehicle.capacity
        model.capacity_constraint = Constraint(model.cars_set, rule=capacity_rule)

        # 5. Subtours eliminados (MTZ constraints)
        def subtour_elimination_rule(model, i, j, k):
            if i != j:
                return model.u[i, k] - model.u[j, k] + len(model.clients_set) * model.x[i, j, k] <= len(model.clients_set) - 1
            else:
                return Constraint.Skip
        model.subtour_elimination = Constraint(model.clients_set, model.clients_set, model.cars_set, rule=subtour_elimination_rule)

        # 6. Prohibir viajes de depósito a depósito
        def no_depot_to_depot_rule(model, i, j, k):
            if i <= len(self.depots) and j <= len(self.depots):
                return model.x[i, j, k] == 0
            else:
                return Constraint.Skip
        model.no_depot_to_depot = Constraint(model.nodes_set, model.nodes_set, model.cars_set, rule=no_depot_to_depot_rule)

        # 7. Restricción de rango
        def range_constraint_rule(model, k):
            vehicle = self.vehicles[k-1]
            total_distance = sum(
                vehicle.calculate_distance(i, j, self.car_matrix_distance, self.drone_matrix_distance) * model.x[i, j, k]
                for i in model.nodes_set for j in model.nodes_set
            )
            return total_distance <= vehicle.range_km
        model.range_constraint = Constraint(model.cars_set, rule=range_constraint_rule)

        # Restricción de capacidad del depósito
        def depot_cap_rule(model, i):
            depot = self.depots[i - 1]  # Asegúrate de que i-1 es un índice válido
            return sum(
                model.y[i, k] for k in model.cars_set
            ) >= depot.capacity
        model.depot_cap_rule = Constraint(model.depots_set, rule=depot_cap_rule)

        # Restricción de demanda del cliente
        def client_demand_rule(model, j):
            return sum(model.z[j, k] for k in model.cars_set) == self.clients[j - len(self.depots) - 1].product
        model.client_demand_rule = Constraint(model.clients_set, rule=client_demand_rule)
        
        # Restricción unir "x" con "y" y "z"
        # Restricción unir "x" con "y" y "z"
        epsilon = 1e-6  # Un valor mínimo positivo
        def unir_rule_y(model, i, j, k):
            return model.y[i, k] >= epsilon * model.x[i, j, k]
        model.unir_rule_y = Constraint(model.depots_set, model.nodes_set, model.cars_set, rule=unir_rule_y)

        def unir_rule_z(model, i, j, k):
            return model.z[j, k] >= epsilon * model.x[i, j, k]
        model.unir_rule_z = Constraint(model.nodes_set, model.clients_set, model.cars_set, rule=unir_rule_z)
        
        def cap_vehiculo(model, i, j, k):
            return sum(model.z[j, k] for k in model.cars_set) <= sum(model.y[i, k] for k in model.cars_set)
        model.cap_vehiculo = Constraint(model.depots_set, model.clients_set, model.cars_set, rule=cap_vehiculo)

        """
        # Restricción de flujo de vehículos
        def flow_rule(model, i, j, k):
            if i in model.depots_set:
                return model.y[i, k] == sum(model.x[i, j, k] * self.clients[j - len(self.depots) - 1].product for j in model.clients_set)
            elif j in model.clients_set:
                return model.z[j, k] == sum(model.x[i, j, k] * self.clients[j - len(self.depots) - 1].product for i in model.depots_set)
            else:
                return Constraint.Skip
        model.flow_rule = Constraint(model.nodes_set, model.nodes_set, model.cars_set, rule=flow_rule)
        """
        """
        print("\nValores de las variables de decisión y:")
        for i in self.model.depots_set:
            for k in self.model.cars_set:
                print(f"y[{i},{k}] = {value(self.model.y[i, k])}")
                
        print("\nValores de las variables de decisión z:")
        for j in self.model.clients_set:
            for k in self.model.cars_set:
                print(f"z[{j},{k}] = {value(self.model.z[j, k])}")       
        """
        
    def solve_model(self):
      #solver = SolverFactory('glpk')
      solver_name = "highs"
      solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
      solver.options['time_limit'] = 1800
      result = solver.solve(self.model, tee=True)
      
      #self.model.pprint()
      self.model.y.display()
      self.model.z.display()

      if (result.solver.status == SolverStatus.ok) and (result.solver.termination_condition == TerminationCondition.optimal):
          print("\nSolución óptima encontrada.")

          # Imprimir el costo total
          total_cost = value(self.model.obj)
          print(f"Costo total: {total_cost:,.2f}\n")

          # Imprimir las rutas y costos de cada vehículo
          for k in self.model.cars_set:
              vehicle = self.vehicles[k-1]
              print(f"Vehículo {k} ({vehicle.name}):")

              # Reconstruir la ruta para el vehículo k
              route = []
              num_depots = len(self.depots)
              print(num_depots)
              vehicle_cost = vehicle.daily_maintenance  # Incluir costo de mantenimiento diario

              # Encontrar el depósito de inicio para el vehículo k
              start_node = None
              for i in self.model.depots_set:
                  for j in self.model.nodes_set:
                      if value(self.model.x[i, j, k]) > 0.5:
                          start_node = j
                          route.append(str(self.index_to_location_id[i]))  # Agregar el depósito inicial
                          break
                  if start_node is not None:
                      break

              if start_node is None:
                  print("  No se encontró una ruta para este vehículo.")
                  continue

              current_node = start_node
              visited_nodes = set()
              visited_nodes.add(current_node)
              route.append(str(self.index_to_location_id[current_node]))

              while True:
                  next_node = None
                  for j in self.model.nodes_set:
                      if value(self.model.x[current_node, j, k]) > 0.5:
                          next_node = j
                          break
                  if next_node is None:
                      break
                  if next_node in visited_nodes:
                      # Verificar si hemos regresado a un depósito
                      if next_node in self.model.depots_set:
                          route.append(str(self.index_to_location_id[next_node]))

                          # Calcular costo del último arco
                          arc_cost = vehicle.calculate_cost(current_node, next_node, 1,self.car_matrix_distance,self.car_matrix_time,self.drone_matrix_distance, num_depots)
                          vehicle_cost += arc_cost
                      break
                  # Agregar nodo a la ruta
                  route.append(str(self.index_to_location_id[next_node]))
                  visited_nodes.add(next_node)

                  # Calcular costo del arco actual
                  arc_cost = vehicle.calculate_cost(current_node, next_node, 1,self.car_matrix_distance,self.car_matrix_time,self.drone_matrix_distance,num_depots)

                  vehicle_cost += arc_cost
                  current_node = next_node

              # Imprimir la ruta
              formatted_route = " -> ".join(route)
              print(f"  Ruta: {formatted_route}")
              print(f"  Costo: {vehicle_cost:,.2f}\n")

      elif result.solver.termination_condition == TerminationCondition.infeasible:
          print("No se encontró una solución factible.")
      else:
          print("El solver terminó con condición:", result.solver.termination_condition)

          # Imprimir el costo total
          total_cost = value(self.model.obj)
          print(f"Costo total: {total_cost:,.2f}\n")

          # Imprimir las rutas y costos de cada vehículo
          for k in self.model.cars_set:
              vehicle = self.vehicles[k-1]
              print(f"Vehículo {k} ({vehicle.name}):")

              # Reconstruir la ruta para el vehículo k
              route = []
              num_depots = len(self.depots)
              vehicle_cost = vehicle.daily_maintenance  # Incluir costo de mantenimiento diario

              # Encontrar el depósito de inicio para el vehículo k
              start_node = None
              for i in self.model.depots_set:
                  for j in self.model.nodes_set:
                      if value(self.model.x[i, j, k]) > 0.5:
                          start_node = j
                          route.append(str(self.index_to_location_id[i]))  # Agregar el depósito inicial
                          break
                  if start_node is not None:
                      break

              if start_node is None:
                  print("  No se encontró una ruta para este vehículo.")
                  continue

              current_node = start_node
              visited_nodes = set()
              visited_nodes.add(current_node)
              route.append(str(self.index_to_location_id[current_node]))

              while True:
                  next_node = None
                  for j in self.model.nodes_set:
                      if value(self.model.x[current_node, j, k]) > 0.5:
                          next_node = j
                          break
                  if next_node is None:
                      break
                  if next_node in visited_nodes:
                      # Verificar si hemos regresado a un depósito
                      if next_node in self.model.depots_set:
                          route.append(str(self.index_to_location_id[next_node]))

                          # Calcular costo del último arco
                          arc_cost = vehicle.calculate_cost(current_node, next_node, 1,self.car_matrix_distance,self.car_matrix_time,self.drone_matrix_distance, num_depots)
                          vehicle_cost += arc_cost
                      break
                  # Agregar nodo a la ruta
                  route.append(str(self.index_to_location_id[next_node]))
                  visited_nodes.add(next_node)

                  # Calcular costo del arco actual
                  arc_cost = vehicle.calculate_cost(current_node, next_node, 1,self.car_matrix_distance,self.car_matrix_time,self.drone_matrix_distance,num_depots)

                  vehicle_cost += arc_cost
                  current_node = next_node

              # Imprimir la ruta
              formatted_route = " -> ".join(route)
              print(f"  Ruta: {formatted_route}")
              print(f"  Costo: {vehicle_cost:,.2f}\n")


## Visualización

In [32]:

import folium
from folium import Marker, PolyLine
from folium.plugins import MarkerCluster
import matplotlib.cm as cm
import matplotlib.colors as colors
import math

def plot_routes_folium(transport_model, vehicles, depots, clients):
    model = transport_model.model  # Acceder al modelo desde transport_model

    # Crear un mapa base centrado en la ubicación promedio de los nodos
    all_coords = [(depot.latitude, depot.longitude) for depot in depots] + \
                 [(client.latitude, client.longitude) for client in clients]
    avg_lat = sum([coord[0] for coord in all_coords]) / len(all_coords)
    avg_lon = sum([coord[1] for coord in all_coords]) / len(all_coords)
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

    # Añadir nodos de depósitos al mapa
    for depot in depots:
      folium.Marker(
          location=[depot.latitude, depot.longitude],
          popup=f"Depósito {depot.location_id}<br>Cap Of:: {depot.capacity}",
          icon=folium.Icon(color='orange', icon='home', prefix='fa')
      ).add_to(m)


    # Añadir nodos de clientes al mapa usando MarkerCluster
    client_cluster = MarkerCluster(name='Clientes').add_to(m)
    for client in clients:
        folium.Marker(
            location=[client.latitude, client.longitude],
            popup=f"Cliente {client.location_id}<br>Demanda: {client.product}",
            icon=folium.Icon(color='blue', icon='user', prefix='fa')
        ).add_to(client_cluster)


    # Generar colores únicos para cada vehículo
    vehicle_colors = {}
    colormap = cm.get_cmap('tab20', len(vehicles))  # Generar colormap con suficientes colores
    for idx, vehicle in enumerate(vehicles):
        rgba = colormap(idx)
        # Convertir RGBA a Hex
        vehicle_colors[vehicle.name] = colors.rgb2hex(rgba)

    # Añadir rutas al mapa
    for k in model.cars_set:
        vehicle = vehicles[k-1]
        route = []
        num_depots = len(depots)

        # Reconstruir la ruta para el vehículo k
        current_node = None
        for i in model.depots_set:
            for j in model.nodes_set:
                if value(model.x[i, j, k]) > 0.5:
                    current_node = i
                    route.append(i)  # Añadir el depósito de inicio a la ruta
                    break
            if current_node is not None:
                break

        if current_node is None:
            continue  # Saltar vehículos sin ruta

        visited_nodes = set()
        visited_nodes.add(current_node)

        # Mientras haya más nodos por visitar
        while True:
            next_node = None
            for j in model.nodes_set:
                if value(model.x[current_node, j, k]) > 0.5 and j not in visited_nodes:
                    next_node = j
                    break

            if next_node is None:
                # Si no hay más nodos clientes por visitar, buscar un depósito de regreso
                for depot_idx in model.depots_set:
                    if value(model.x[current_node, depot_idx, k]) > 0.5:
                        route.append(depot_idx)
                        current_node = depot_idx
                        break
                break  # Salir del bucle cuando se regresa a un depósito

            # Añadir el siguiente nodo a la ruta
            route.append(next_node)
            visited_nodes.add(next_node)
            current_node = next_node

        # Convertir índices de nodos a coordenadas
        route_coords = []
        for node_idx in route:
            location_id = transport_model.index_to_location_id[node_idx]
            # Buscar la latitud y longitud correspondientes
            if node_idx <= len(depots):
                # Es un depósito
                node = next((d for d in depots if d.location_id == location_id), None)
            else:
                # Es un cliente
                node = next((c for c in clients if c.location_id == location_id), None)
            if node:
                route_coords.append((node.latitude, node.longitude))

        # Añadir la ruta al mapa con direccionalidad al final
        for i in range(len(route_coords) - 1):
            start = route_coords[i]
            end = route_coords[i + 1]

            # Dibujar la línea entre los puntos
            folium.PolyLine(
                locations=[start, end],
                color=vehicle_colors[vehicle.name],
                weight=5,
                opacity=0.8,
                tooltip=vehicle.name
            ).add_to(m)

            # Calcular el ángulo de la flecha
            angle = math.atan2(end[0] - start[0], end[1] - start[1]) * 180 / math.pi

            # Añadir una flecha al final del segmento para indicar la dirección
            folium.RegularPolygonMarker(
                location=[end[0] - 0.0001 * (end[0] - start[0]), end[1] - 0.0001 * (end[1] - start[1])],
                color=vehicle_colors[vehicle.name],
                fill_color=vehicle_colors[vehicle.name],
                number_of_sides=3,
                radius=8,
                rotation=angle
            ).add_to(m)

    # Añadir leyenda de vehículos
    legend_html = '''
     <div style="position: fixed;
                 bottom: 50px; left: 50px; width: 200px; height: 300px;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 overflow: auto;
                 ">
                 &nbsp;<b>Leyenda de Vehículos</b><br>
    '''
    for vehicle_name, color in vehicle_colors.items():
        legend_html += f'''
        &nbsp;<i style="background:{color};width:10px;height:10px;display:inline-block;"></i>
        &nbsp;{vehicle_name}<br>
        '''
    legend_html += '</div>'

    m.get_root().html.add_child(folium.Element(legend_html))

    # Añadir control de capas
    folium.LayerControl().add_to(m)

    # Guardar el mapa en un archivo HTML
    m.save('rutas_vehiculos.html')

    return m


## Correr caso

In [84]:
def main():

    # Rutas de archivos CSV
    path = '../Data/case_3_supply_limits/'
    depots_csv_path = path + 'Depots.csv'
    depot_capacities_csv_path = path + 'DepotCapacities.csv'
    clients_csv_path = path + 'Clients.csv'
    vehicles_csv_path = path + 'Vehicles.csv'

    # Leer archivos CSV
    depots_df = pd.read_csv(depots_csv_path)
    depot_capacities_df = pd.read_csv(depot_capacities_csv_path)
    clients_df = pd.read_csv(clients_csv_path)
    vehicles = load_vehicles(vehicles_csv_path)
    # Verificar la cantidad de vehículos cargados
    print(f"Número de vehículos cargados: {len(vehicles)}")

    # Validar datos
    try:
        validate_data(depots_df, clients_df,depot_capacities_df)
    except ValueError as ve:
        print(f"Error de validación de datos: {ve}")
        return  # Usar 'return' en lugar de 'exit()' en Jupyter

    # Crear instancias de Depots y Clients
    depots = load_depots_with_capacity(depots_csv_path, depot_capacities_csv_path)
    clients = [Client(location_id=row['LocationID'], longitude=row['Longitude'], latitude=row['Latitude'],
                      product=row['Product'])
               for index, row in clients_df.iterrows()]

    # Instanciar el modelo de transporte
    transport_model = TransportationModel(vehicles=vehicles, depots=depots, clients=clients)

    # Construir matrices de distancia y tiempo
    transport_model.build_distance_matrices()

    # Construir el modelo de optimización
    transport_model.build_model()
    transport_model.solve_model()

    # Visualizar las rutas
    m = plot_routes_folium(transport_model, vehicles, depots, clients)
    from IPython.display import display
    display(m)

main()


Tipo de vehículo desconocido: EV. Se omite.
Tipo de vehículo desconocido: EV. Se omite.
Número de vehículos cargados: 4
Cliente 1: Client(location_id=13.0, longitude=-74.19699184741948, latitude=4.632552840424734, product=12.0)
Depots_set definido: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
clients_set
HiGHS 1.8.1:   lim:time = 1800
y : Size=48, Index=depots_set*cars_set
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
     (1, 1) :     0 :  15.0 :  None : False : False : NonNegativeReals
     (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
     (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
     (1, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
     (2, 1) :     0 :  15.0 :  None : False : False : NonNegativeReals
     (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
     (2, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
     (2, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals


C:\Users\Asuar\AppData\Local\Temp\ipykernel_3548\1315555029.py:39: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('tab20', len(vehicles))  # Generar colormap con suficientes colores
